In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
from bs4 import BeautifulSoup
import time
import re

options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--lang=ko_KR")

service = Service()
driver = webdriver.Chrome(service=service, options=options)
driver.implicitly_wait(10)
driver.maximize_window()

wait = WebDriverWait(driver, 10)
action = ActionChains(driver)

urls = [
    "https://www.youtube.com/watch?v=zA-F9EtsRJc&list=PLuMuHAJh9g_Py_PSm8gmHdlcil6CQ9QCM",
    "https://www.youtube.com/watch?v=jpKAf-y3psg&list=PLWoGc25qzA3tNkDJ6_RQzsTemA4RFcScb",
    "https://www.youtube.com/watch?v=bWN3ZIV0BPQ&list=PLWoGc25qzA3vthfzjd7LAS24_A58iIKZv",
    "https://www.youtube.com/watch?v=0xAHCaHWbKk&list=PLWoGc25qzA3s__HwGKJphyLl_KZTEwYGS",
    "https://www.youtube.com/watch?v=rUFJ62QhMZM&list=PLWoGc25qzA3vAvOy52DJN7vco_jkU0cV2",
    "https://www.youtube.com/watch?v=dOGpXEOiYH8&list=PLWoGc25qzA3vRoh0umDUCk3FnWjk6IKhF",
    "https://www.youtube.com/watch?v=MzPvmiSKcj8&list=PLWoGc25qzA3s5wC1AdhHf1LGojWrE8pOa",
    "https://www.youtube.com/watch?v=pOV-txFfYDE&list=PLWoGc25qzA3uNHuMdjiRyYJVVY-_Yq3TA",
    "https://www.youtube.com/watch?v=NSuwkeG_ro0&list=PLWoGc25qzA3tSRZKQVLoE7NEcUfQFaMLN",
    "https://www.youtube.com/watch?v=EFB18WyD1G0&list=PLWoGc25qzA3tAoQ3-7uAt7ZywMAMwbPPK"
]
prev_idx = 0
datas = list()
for url in urls:
    driver.get(url)
    time.sleep(0.5)

    playlists = driver.find_element(
        By.XPATH,
        "/html/body/ytd-app/div[1]/ytd-page-manager/ytd-watch-flexy/div[5]/div[2]/div/ytd-playlist-panel-renderer/div/div[3]",
    )
    title_elems = playlists.find_elements(By.ID, "video-title")
    for title in title_elems:
        # title = title.text.split("|")[-1]
        title = title.text
        datas.append([title])

    time.sleep(3)
    run_time = playlists.find_elements(By.CLASS_NAME, "badge-shape-wiz__text")
    print(len(run_time))
    for i, r_time in enumerate(run_time):
        i = i + prev_idx
        r_time = r_time.text
        datas[i].append(r_time)

    link_imgs = playlists.find_elements(By.CSS_SELECTOR, "img")
    for i, link in enumerate(link_imgs):
        action.move_to_element(link).perform()
        link.click()

        time.sleep(1)
        try:
            premium = driver.find_element(By.XPATH,'//*[@id="dismiss-button"]/yt-button-shape/button/yt-touch-feedback-shape/div/div[2]')
            premium.click()
        except Exception as e:
            pass
        likes = driver.find_element(
            By.XPATH,
            '//*[@id="top-level-buttons-computed"]/segmented-like-dislike-button-view-model/yt-smartimation/div/div/like-button-view-model/toggle-button-view-model/button-view-model/button/div[2]',
        )
        likes = likes.text  # 1000 단위는 K 로 나타나 정확한 수치를 알 수 없음
        datas[i].append(likes)

        expand_button = driver.find_element(By.ID, "expand")
        expand_button.click()
        # 조회수
        views = driver.find_element(By.XPATH, '//*[@id="info"]/span[1]')
        views = int(re.sub(r"[가-힣,\s]", "", views.text))
        datas[i].append(views)
        # DATETIME
        date = driver.find_element(By.XPATH, '//*[@id="info"]/span[3]')
        date = re.sub(r"[가-힣,.:]", "", date.text).strip().split()
        for d in date:
            datas[i].append(d)

        button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable(
                (
                    By.XPATH,
                    '//*[@id="primary-button"]/ytd-button-renderer/yt-button-shape/button',
                )
            )
        )
        action.move_to_element(button).perform()
        button.click()

        wait.until(EC.presence_of_element_located((By.ID, "secondary")))
        time.sleep(3)

        # 페이지 소스 가져오기
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")

        # 자막 세그먼트 모두 가져오기
        segments = soup.find_all(
            "div", class_="segment style-scope ytd-transcript-segment-renderer"
        )

        # 자막 시간과 텍스트 추출
        texts = list()

        for segment in segments:
            text = segment.find(
                "yt-formatted-string",
                class_="segment-text style-scope ytd-transcript-segment-renderer",
            ).get_text(strip=True)
            texts.append(text)

        datas[i].append(texts)
        prev_idx += len(run_time)


driver.quit()

153


In [6]:
print(datas[0])

['EP.1 을지로 | 초계무침에 막국수 사리는 넣어줘야 배우신 분 | 더밥스튜디오 | 최자로드']


In [5]:
import pandas as pd

df = pd.DataFrame(
    datas,
    columns=["title", "run_time", "likes", "views", "year", "month", "day", "script"],
)
print(df.tail(4))

ValueError: 8 columns passed, passed data had 1 columns

In [11]:
# SAVE
df.to_csv('result.csv',index=False)